In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt

import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# The Python magic
import IPython
import signal

def interrupted(_interrupted=[False], _default=[None]):
    if _default[0] is None or signal.getsignal(signal.SIGINT) == _default[0]:
        _interrupted[0] = False
        def handle(signal, frame):
            if _interrupted[0] and _default[0] is not None:
                _default[0](signal, frame)
            print('Interrupt!')
            _interrupted[0] = True
        _default[0] = signal.signal(signal.SIGINT, handle)
    return _interrupted[0]

def enumerate_cycle(g):
    epoch = 0
    while True:
        for i,x in enumerate(g):
            yield (epoch,i), x
        epoch = epoch + 1

In [5]:
# Loading the dataset
pd.set_option('display.max_columns', 500)

diabetic = pd.read_csv('diabetes/diabetic_data_original.csv')
diabetic

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,MC,?,51,0,16,0,0,0,250.13,291,458,9,None,>8,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,MC,?,33,3,18,0,0,1,560,276,787,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,MC,?,53,0,9,1,0,0,38,590,296,13,None,None,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,MC,Surgery-General,45,2,21,0,0,1,996,285,998,9,None,None,No,No,No,No,No,No,Steady,No,No,Steady,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO


In [4]:
patient_nbr_counts = diabetic['patient_nbr'].value_counts()
df_diabetic = diabetic[diabetic['patient_nbr'].isin(patient_nbr_counts.index[patient_nbr_counts.gt(1)])].sort_values(['patient_nbr', 'encounter_id'])
pd.set_option('display.max_rows', 25)
print(df_diabetic[['encounter_id', 'patient_nbr', 'readmitted']])

        encounter_id  patient_nbr readmitted
4267        24437208          135        <30
4780        26264286          135        >30
1164         8380170         1152        >30
5953        30180318         1152        >30
14180       55533660         1152        >30
...              ...          ...        ...
99544      414418886    188634893         NO
93050      330256946    188970179        <30
101595     441488168    188970179         NO
90933      302575430    189257846         NO
98191      397823084    189257846        >30

[47021 rows x 3 columns]


In [7]:
df_diabetic = df_diabetic.set_index(df_diabetic['encounter_id'])
df_diabetic['len_next_stay'] = pd.Series()
for patient in df_diabetic['patient_nbr'].unique():
    encounter_ids = list(df_diabetic[df_diabetic['patient_nbr'] == patient]['encounter_id'])
    readmitted = df_diabetic.loc[encounter_ids, ['readmitted']]['readmitted'].value_counts()
    lengths_of_stay = df_diabetic.loc[encounter_ids, 'time_in_hospital']
    df_diabetic.loc[encounter_ids[:-1], ['len_next_stay']] = lengths_of_stay.values[1:]

df_readmitted_only = df_diabetic.copy()
df_readmitted_only = df_readmitted_only[np.isfinite(df_readmitted_only['len_next_stay'])]

In [10]:
np.unique(df_readmitted_only['patient_nbr'].value_counts().values, return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 27, 39]),
 array([10434,  3328,  1421,   717,   346,   207,   111,    70,    42,
           20,    19,    14,     5,     9,     4,     3,     6,     3,
            6,     1,     2,     3,     1,     1]))